# Module 01: AWS SageMaker Basics

**Difficulty**: ⭐⭐ Intermediate

**Estimated Time**: 60 minutes

**Prerequisites**: 
- Module 00: Introduction to Cloud ML Services
- AWS account (free tier)
- Basic Python and ML knowledge

## Learning Objectives

By the end of this notebook, you will be able to:
1. Set up AWS credentials and configure the SageMaker SDK
2. Understand SageMaker architecture and key components
3. Use SageMaker Studio and notebook instances
4. Train a simple ML model using SageMaker's built-in algorithms
5. Manage training jobs and monitor costs
6. Work with S3 for data storage and model artifacts

## 1. AWS SageMaker Overview

### What is Amazon SageMaker?

Amazon SageMaker is a comprehensive ML platform that covers the entire ML workflow:

**Key Components**:
1. **SageMaker Studio**: Integrated development environment (IDE) for ML
2. **Notebook Instances**: Managed Jupyter notebooks
3. **Training**: Distributed training with auto-scaling
4. **Deployment**: Hosting for real-time and batch inference
5. **Pipelines**: Workflow automation and MLOps
6. **Model Registry**: Centralized model catalog

### SageMaker Architecture

```
┌─────────────────────────────────────────────────────────┐
│                  SageMaker Studio/IDE                   │
│  (Development Environment - Notebooks, Experiments)     │
└─────────────────────────────────────────────────────────┘
                          |
                          v
┌─────────────────────────────────────────────────────────┐
│                     Amazon S3                           │
│  (Data Storage - Training Data, Model Artifacts)        │
└─────────────────────────────────────────────────────────┘
                          |
                          v
┌─────────────────────────────────────────────────────────┐
│                  Training Jobs                          │
│  (Managed Compute - ml.m5.xlarge, ml.p3.2xlarge, etc.)  │
└─────────────────────────────────────────────────────────┘
                          |
                          v
┌─────────────────────────────────────────────────────────┐
│                  Model Registry                         │
│         (Versioned Model Artifacts)                     │
└─────────────────────────────────────────────────────────┘
                          |
                          v
┌─────────────────────────────────────────────────────────┐
│           Endpoints (Real-time Inference)               │
│        or Batch Transform (Batch Inference)             │
└─────────────────────────────────────────────────────────┘
```

## 2. Setup and Configuration

### 2.1 Install Required Libraries

⚠️ **Note**: This notebook demonstrates SageMaker usage. Some cells are for reference only and won't execute without valid AWS credentials.

In [ ]:
# Install SageMaker SDK
# Uncomment to install (already in requirements.txt)
# !pip install sagemaker boto3 --upgrade

import sys
print(f"Python version: {sys.version}")

# Import required libraries
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# Display library versions
print(f"\nSageMaker SDK version: {sagemaker.__version__}")
print(f"Boto3 version: {boto3.__version__}")

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

### 2.2 AWS Credentials Configuration

**Before using SageMaker, you need AWS credentials**:

1. **Create AWS Account**: Sign up at [aws.amazon.com](https://aws.amazon.com)
2. **Create IAM User**: 
   - Go to IAM Console
   - Create user with programmatic access
   - Attach policy: `AmazonSageMakerFullAccess`
   - Save Access Key ID and Secret Access Key

3. **Configure Credentials** (choose one method):

**Method 1: AWS CLI (Recommended)**
```bash
# Install AWS CLI
pip install awscli

# Configure credentials
aws configure
# Enter:
#   - AWS Access Key ID
#   - AWS Secret Access Key
#   - Default region (e.g., ap-southeast-1 for Singapore)
#   - Default output format (json)
```

**Method 2: Environment Variables**
```python
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'your-access-key-id'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'your-secret-access-key'
os.environ['AWS_DEFAULT_REGION'] = 'ap-southeast-1'
```

**Method 3: Credentials File**
```bash
# Create ~/.aws/credentials file:
[default]
aws_access_key_id = YOUR_ACCESS_KEY
aws_secret_access_key = YOUR_SECRET_KEY
region = ap-southeast-1
```

⚠️ **Security Warning**: NEVER commit credentials to Git! Use `.env` files or AWS Secrets Manager for production.

In [ ]:
# Simulated SageMaker session setup (for demonstration)
# In real usage, this would connect to AWS

class MockSageMakerSession:
    """
    Mock SageMaker session for demonstration without AWS credentials.
    Replace with real session when you have AWS access.
    """
    def __init__(self):
        self.region = 'ap-southeast-1'  # Singapore region
        self.account_id = '123456789012'  # Mock account ID
        
    def default_bucket(self):
        return f"sagemaker-{self.region}-{self.account_id}"
    
    def get_role(self):
        return f"arn:aws:iam::{self.account_id}:role/SageMakerExecutionRole"

# For demonstration purposes
DEMO_MODE = True  # Set to False when using real AWS credentials

if DEMO_MODE:
    # Mock session for learning
    print("🎓 Running in DEMO MODE (no AWS credentials required)")
    print("   Set DEMO_MODE = False when you have AWS credentials\n")
    
    mock_session = MockSageMakerSession()
    region = mock_session.region
    bucket = mock_session.default_bucket()
    role = mock_session.get_role()
else:
    # Real SageMaker session
    print("🔑 Connecting to AWS SageMaker...\n")
    
    # Create SageMaker session
    sess = sagemaker.Session()
    
    # Get execution role
    # This role allows SageMaker to access other AWS services on your behalf
    role = sagemaker.get_execution_role()
    
    # Get default S3 bucket (created automatically by SageMaker)
    bucket = sess.default_bucket()
    
    # Get region
    region = sess.boto_region_name

print(f"Region: {region}")
print(f"S3 Bucket: {bucket}")
print(f"Execution Role: {role}")
print(f"\n✅ SageMaker session configured successfully!")

## 3. Working with S3 for Data Storage

### Understanding S3 in SageMaker

Amazon S3 (Simple Storage Service) is AWS's object storage service. SageMaker uses S3 for:
- **Input Data**: Training and validation datasets
- **Output Data**: Model artifacts, predictions
- **Model Storage**: Trained model files

**S3 URI Format**: `s3://bucket-name/prefix/key`

**Free Tier**: 
- 5 GB of S3 Standard storage
- 20,000 GET requests
- 2,000 PUT requests
- Valid for 12 months

In [ ]:
# Create sample training data
# Using Iris dataset for classification example

from sklearn.datasets import load_iris
from io import StringIO

# Load iris dataset
iris = load_iris()
iris_data = pd.DataFrame(
    data=np.c_[iris['data'], iris['target']],
    columns=iris['feature_names'] + ['target']
)

print("Iris Dataset Sample:")
print(iris_data.head())
print(f"\nShape: {iris_data.shape}")
print(f"Classes: {iris.target_names}")

# Split data
train_data, test_data = train_test_split(
    iris_data, 
    test_size=0.2, 
    random_state=42, 
    stratify=iris_data['target']
)

print(f"\nTraining set: {len(train_data)} samples")
print(f"Test set: {len(test_data)} samples")

In [ ]:
# Prepare data for SageMaker (CSV format without header, target as first column)
# This is the format expected by many SageMaker built-in algorithms

def prepare_sagemaker_data(df, target_column='target'):
    """
    Prepare DataFrame for SageMaker format.
    
    SageMaker expects:
    - No header row
    - Target variable as first column
    - CSV format
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input DataFrame
    target_column : str
        Name of target column
    
    Returns:
    --------
    pd.DataFrame : Reformatted DataFrame
    """
    # Move target column to first position
    cols = [target_column] + [col for col in df.columns if col != target_column]
    return df[cols]

# Prepare data
train_formatted = prepare_sagemaker_data(train_data)
test_formatted = prepare_sagemaker_data(test_data)

# Save locally first
train_file = 'iris_train.csv'
test_file = 'iris_test.csv'

train_formatted.to_csv(train_file, index=False, header=False)
test_formatted.to_csv(test_file, index=False, header=False)

print(f"✅ Data saved locally:")
print(f"   - {train_file}")
print(f"   - {test_file}")
print(f"\nSample of formatted training data (first 3 rows):")
print(train_formatted.head(3).to_string(index=False))

In [ ]:
# Upload data to S3 (simulated in demo mode)

if DEMO_MODE:
    # Simulate S3 upload
    train_s3_uri = f"s3://{bucket}/iris/train/iris_train.csv"
    test_s3_uri = f"s3://{bucket}/iris/test/iris_test.csv"
    
    print("📦 [SIMULATED] Uploading data to S3...")
    print(f"   Training data would be at: {train_s3_uri}")
    print(f"   Test data would be at: {test_s3_uri}")
    print("\n💡 In real usage, data is uploaded to S3 for SageMaker access")
else:
    # Real S3 upload
    prefix = 'iris'
    
    # Upload training data
    train_s3_uri = sess.upload_data(
        path=train_file,
        bucket=bucket,
        key_prefix=f'{prefix}/train'
    )
    
    # Upload test data
    test_s3_uri = sess.upload_data(
        path=test_file,
        bucket=bucket,
        key_prefix=f'{prefix}/test'
    )
    
    print("✅ Data uploaded to S3:")
    print(f"   Training: {train_s3_uri}")
    print(f"   Test: {test_s3_uri}")

# Store URIs for later use
data_locations = {
    'train': train_s3_uri,
    'test': test_s3_uri
}

print("\n📍 S3 Data Locations saved")

## 4. SageMaker Built-in Algorithms

### What are Built-in Algorithms?

SageMaker provides optimized, pre-built algorithms that you can use without writing training code:

**Classification/Regression**:
- **XGBoost**: Gradient boosting (most popular)
- **Linear Learner**: Linear models with SGD
- **K-NN**: K-nearest neighbors

**Computer Vision**:
- **Image Classification**: CNN-based classification
- **Object Detection**: Detect objects in images
- **Semantic Segmentation**: Pixel-level classification

**NLP**:
- **BlazingText**: Text classification and word embeddings
- **Seq2Seq**: Sequence-to-sequence (translation, summarization)

**Other**:
- **Factorization Machines**: Recommendation systems
- **K-Means**: Clustering
- **PCA**: Dimensionality reduction

**Advantages**:
- No training code needed
- Optimized for distributed training
- Automatic hyperparameter tuning available
- Well-documented and tested

## 5. Training with SageMaker XGBoost

Let's train a model using SageMaker's XGBoost algorithm:

### Key Concepts:

1. **Container Image**: Pre-built Docker image with algorithm
2. **Estimator**: Configuration for training job
3. **Training Job**: Managed compute that trains your model
4. **Hyperparameters**: Algorithm-specific parameters

In [ ]:
# Get XGBoost container image URI
# SageMaker maintains optimized algorithm containers in ECR

if DEMO_MODE:
    # Simulated container URI
    container = f"683313688378.dkr.ecr.{region}.amazonaws.com/sagemaker-xgboost:1.7-1"
    print("📦 [SIMULATED] XGBoost Container URI:")
    print(f"   {container}")
else:
    # Get real container URI
    from sagemaker import image_uris
    
    container = image_uris.retrieve(
        framework='xgboost',
        region=region,
        version='1.7-1'  # XGBoost version
    )
    print(f"✅ XGBoost Container: {container}")

print("\n💡 This container has XGBoost pre-installed and optimized for SageMaker")

In [ ]:
# Configure the XGBoost estimator
# This defines how the training job will run

if DEMO_MODE:
    # Show configuration without creating actual estimator
    estimator_config = {
        'image_uri': container,
        'role': role,
        'instance_count': 1,
        'instance_type': 'ml.m5.xlarge',  # Free tier eligible
        'volume_size': 5,  # GB
        'max_run': 3600,  # Max training time in seconds
        'output_path': f's3://{bucket}/iris/output',
        'sagemaker_session': 'mock_session'
    }
    
    print("⚙️  [SIMULATED] XGBoost Estimator Configuration:")
    print(json.dumps(estimator_config, indent=2))
    
    # Simulated hyperparameters
    hyperparameters = {
        'objective': 'multi:softmax',  # Multi-class classification
        'num_class': 3,  # 3 iris species
        'num_round': 100,  # Number of boosting rounds
        'max_depth': 5,
        'eta': 0.2,  # Learning rate
        'subsample': 0.8,
        'colsample_bytree': 0.8
    }
    
    print("\n📊 Hyperparameters:")
    print(json.dumps(hyperparameters, indent=2))
    
else:
    # Real estimator creation
    from sagemaker.estimator import Estimator
    
    xgb_estimator = Estimator(
        image_uri=container,
        role=role,
        instance_count=1,
        instance_type='ml.m5.xlarge',  # Free tier eligible (first 2 months)
        volume_size=5,  # EBS volume size in GB
        max_run=3600,  # Maximum training time (1 hour)
        output_path=f's3://{bucket}/iris/output',
        sagemaker_session=sess
    )
    
    # Set hyperparameters
    xgb_estimator.set_hyperparameters(
        objective='multi:softmax',
        num_class=3,
        num_round=100,
        max_depth=5,
        eta=0.2,
        subsample=0.8,
        colsample_bytree=0.8
    )
    
    print("✅ XGBoost Estimator created and configured")

print("\n💰 COST ESTIMATE:")
print("   Instance: ml.m5.xlarge at ~$0.23/hour")
print("   Expected training time: <5 minutes")
print("   Estimated cost: ~$0.02 per training job")
print("\n⚠️  Remember to check current pricing in your region!")

In [ ]:
# Launch training job

if DEMO_MODE:
    print("🎯 [SIMULATED] Training Job Launch")
    print("\nIn real usage, this would:")
    print("  1. Spin up ml.m5.xlarge instance")
    print("  2. Download training data from S3")
    print("  3. Train XGBoost model")
    print("  4. Save model artifacts to S3")
    print("  5. Terminate instance")
    print("\nExpected output:")
    print("  - Training job name: xgboost-iris-2025-11-19-12-34-56")
    print("  - Status: InProgress → Completed")
    print("  - Training time: ~3-5 minutes")
    print("  - Model location: s3://{bucket}/iris/output/...")
    
    # Simulate training metrics
    simulated_metrics = pd.DataFrame({
        'iteration': range(0, 101, 10),
        'train_mlogloss': [1.0986, 0.7234, 0.5123, 0.3891, 0.3012, 
                           0.2456, 0.2098, 0.1834, 0.1645, 0.1512, 0.1423]
    })
    
    print("\n📊 Simulated Training Metrics:")
    print(simulated_metrics.to_string(index=False))
    
else:
    # Real training job
    print("🚀 Launching SageMaker training job...\n")
    
    # Start training
    xgb_estimator.fit({
        'train': train_s3_uri,
        'validation': test_s3_uri
    })
    
    print("\n✅ Training job completed!")
    print(f"Model artifacts saved to: {xgb_estimator.model_data}")

print("\n💡 Training Job Tips:")
print("   • Monitor in SageMaker Console → Training Jobs")
print("   • Check CloudWatch for detailed logs")
print("   • Training job automatically terminates when done")
print("   • Model artifacts are saved to S3 automatically")

## 6. Cost Management and Monitoring

### Understanding SageMaker Costs

**Cost Components**:
1. **Notebook Instances**: Charged per hour (can be stopped)
2. **Training Jobs**: Charged per second (minimum 1 minute)
3. **Endpoints**: Charged per hour (24/7 if not deleted)
4. **Storage**: S3 charges for data and model storage

**Cost Optimization Tips**:

In [ ]:
# Cost estimation helper functions

def estimate_training_cost(instance_type, training_time_minutes, region='us-east-1'):
    """
    Estimate SageMaker training job cost.
    
    Parameters:
    -----------
    instance_type : str
        EC2 instance type (e.g., 'ml.m5.xlarge')
    training_time_minutes : float
        Expected training time in minutes
    region : str
        AWS region
    
    Returns:
    --------
    dict : Cost breakdown
    """
    # Approximate hourly rates (as of 2025, US East)
    # Always check current pricing!
    hourly_rates = {
        'ml.t3.medium': 0.05,   # Free tier eligible
        'ml.m5.large': 0.115,
        'ml.m5.xlarge': 0.23,
        'ml.m5.2xlarge': 0.46,
        'ml.p3.2xlarge': 3.825,  # GPU
        'ml.p3.8xlarge': 14.688, # 4x GPU
    }
    
    if instance_type not in hourly_rates:
        return {'error': f'Unknown instance type: {instance_type}'}
    
    hourly_rate = hourly_rates[instance_type]
    training_hours = training_time_minutes / 60
    training_cost = hourly_rate * training_hours
    
    # Add storage cost (minimal for small jobs)
    storage_cost = 0.01  # Approximate
    
    total_cost = training_cost + storage_cost
    
    return {
        'instance_type': instance_type,
        'hourly_rate': hourly_rate,
        'training_time_minutes': training_time_minutes,
        'training_cost': round(training_cost, 4),
        'storage_cost': round(storage_cost, 4),
        'total_cost': round(total_cost, 4)
    }

# Example cost estimates
scenarios = [
    ('ml.m5.xlarge', 5),    # Small model, quick training
    ('ml.m5.xlarge', 60),   # Medium model
    ('ml.p3.2xlarge', 30),  # GPU training
]

print("💰 TRAINING COST ESTIMATES (USD)")
print("=" * 80)

cost_results = []
for instance, minutes in scenarios:
    cost = estimate_training_cost(instance, minutes)
    cost_results.append(cost)
    print(f"\nScenario: {instance} for {minutes} minutes")
    print(f"  Hourly Rate: ${cost['hourly_rate']}")
    print(f"  Training Cost: ${cost['training_cost']}")
    print(f"  Total Cost: ${cost['total_cost']}")

print("\n" + "=" * 80)
print("\n⚠️  IMPORTANT COST TIPS:")
print("   1. Use Managed Spot Training for 70% cost savings")
print("   2. Stop notebook instances when not in use")
print("   3. Delete endpoints after testing")
print("   4. Use AWS Cost Explorer for tracking")
print("   5. Set up billing alerts immediately!")

In [ ]:
# Visualize cost comparison

instance_types = ['ml.t3.medium', 'ml.m5.xlarge', 'ml.m5.2xlarge', 'ml.p3.2xlarge']
hourly_costs = [0.05, 0.23, 0.46, 3.825]

# Calculate costs for 1 hour of training
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Cost comparison
colors = ['green', 'blue', 'orange', 'red']
bars = ax1.bar(range(len(instance_types)), hourly_costs, color=colors, alpha=0.7)
ax1.set_xticks(range(len(instance_types)))
ax1.set_xticklabels(instance_types, rotation=45, ha='right')
ax1.set_ylabel('Cost per Hour (USD)', fontsize=11)
ax1.set_title('SageMaker Training Instance Costs', fontsize=12, fontweight='bold')
ax1.set_yscale('log')  # Log scale due to wide range
ax1.grid(axis='y', alpha=0.3)

# Add cost labels
for i, (bar, cost) in enumerate(zip(bars, hourly_costs)):
    ax1.text(i, cost, f'${cost}', ha='center', va='bottom', fontsize=9)

# Monthly cost if running 24/7 (what NOT to do!)
monthly_costs = [cost * 24 * 30 for cost in hourly_costs]
bars2 = ax2.bar(range(len(instance_types)), monthly_costs, color=colors, alpha=0.7)
ax2.set_xticks(range(len(instance_types)))
ax2.set_xticklabels(instance_types, rotation=45, ha='right')
ax2.set_ylabel('Monthly Cost if 24/7 (USD)', fontsize=11)
ax2.set_title('⚠️ Cost if Endpoint Runs 24/7 (DON\'T DO THIS!)', 
              fontsize=12, fontweight='bold', color='red')
ax2.grid(axis='y', alpha=0.3)

# Add cost labels
for i, (bar, cost) in enumerate(zip(bars2, monthly_costs)):
    ax2.text(i, cost, f'${cost:.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n💡 KEY INSIGHT:")
print("   A ml.p3.2xlarge endpoint running 24/7 costs $2,754/month!")
print("   Always delete endpoints when not in use.")
print("   Use on-demand endpoints or batch transform for production.")

## 7. Best Practices for SageMaker

### Development Workflow

**1. Local Development First**:
```python
# Develop and test locally
# → Use SageMaker local mode for testing
# → Move to cloud when ready
```

**2. Use Managed Spot Training**:
- Save up to 70% on training costs
- Good for interruptible workloads
- Enable with `use_spot_instances=True`

**3. Version Everything**:
- Track experiments with SageMaker Experiments
- Version datasets in S3
- Use Model Registry for model versioning

**4. Monitor Costs**:
- Set up AWS Budgets alerts
- Tag all resources
- Review Cost Explorer weekly

**5. Security**:
- Use IAM roles (never hard-code credentials)
- Enable encryption for S3 and EBS
- Use VPC for network isolation
- Enable CloudTrail for auditing

## Exercises

### Exercise 1: Data Preparation

Prepare the California Housing dataset for SageMaker training:
1. Load the dataset
2. Split into train/test
3. Format for SageMaker (target first, no header)
4. Save as CSV files

In [ ]:
# Your solution here
# Hint: Use sklearn.datasets.fetch_california_housing()



### Exercise 2: Cost Calculation

You need to train 100 models for hyperparameter tuning. Each training job takes 10 minutes on ml.m5.xlarge.

Calculate:
1. Total training time
2. Cost with regular instances
3. Cost with Managed Spot Training (70% discount)
4. Total savings

In [ ]:
# Your solution here



### Exercise 3: Configuration Comparison

Compare two training configurations:
- **Config A**: ml.m5.2xlarge for 30 minutes
- **Config B**: ml.p3.2xlarge for 10 minutes (GPU speeds up training)

Which is more cost-effective? What other factors should you consider besides cost?

In [ ]:
# Your solution here



### Exercise 4: S3 Path Construction

Write a function that constructs proper S3 paths for a SageMaker project with the following structure:
```
s3://my-bucket/
  └── projects/
      └── project-name/
          ├── data/
          │   ├── train/
          │   └── test/
          ├── models/
          └── outputs/
```

In [ ]:
# Your solution here
def create_s3_paths(bucket, project_name):
    """
    Create S3 path structure for a SageMaker project.
    
    Parameters:
    -----------
    bucket : str
        S3 bucket name
    project_name : str
        Project name
    
    Returns:
    --------
    dict : Dictionary of S3 paths
    """
    # Your code here
    pass

# Test your function
# paths = create_s3_paths('my-ml-bucket', 'customer-churn')
# print(paths)


## Summary

### Key Takeaways

1. **SageMaker Architecture**:
   - Studio/Notebooks for development
   - S3 for data and model storage
   - Training jobs on managed compute
   - Endpoints for inference

2. **Built-in Algorithms**:
   - XGBoost, Linear Learner, K-NN for tabular data
   - Image Classification, Object Detection for vision
   - BlazingText for NLP
   - No training code required!

3. **Data Management**:
   - Use S3 for all data storage
   - Format data correctly (target first, no header)
   - Organize with clear prefixes

4. **Cost Management**:
   - Training: Billed per second
   - Endpoints: Billed per hour (delete when not needed!)
   - Use Managed Spot Training for savings
   - Monitor with Cost Explorer

5. **Best Practices**:
   - Develop locally, train in cloud
   - Version everything (data, code, models)
   - Set up billing alerts
   - Use IAM roles for security

### What's Next?

In **Module 02: AWS SageMaker Deployment**, you'll learn:
- Deploy models to real-time endpoints
- Use Batch Transform for large-scale inference
- Implement A/B testing
- Monitor production models
- Auto-scaling and cost optimization

### Additional Resources

- [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/)
- [SageMaker Examples](https://github.com/aws/amazon-sagemaker-examples)
- [AWS Machine Learning Blog](https://aws.amazon.com/blogs/machine-learning/)
- [SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/)
- [AWS Free Tier](https://aws.amazon.com/free/)

---

**Next Steps**: 
1. Set up your AWS account if you haven't already
2. Configure AWS credentials
3. Try running a simple training job
4. Proceed to Module 02 for deployment

**Remember**: Always check your AWS billing dashboard and set up alerts before running any jobs!